STEP 1


In [1]:
import pandas as pd
df = pd.read_csv("data.csv")
df

,SrNo,Patient_Id,Fever_Mild,Fever_Low,Fever_High,DOB,Cough_Mild,Cough_Low,Cough_High,Cold_Mild,Cold_Low,Cold_High,Other_Symptoms
0,1,1,Y,N,N,02-06-1997,N,Y,Y,N,N,Y,Body Ache
1,2,2,Y,N,N,28-06-2017,N,Y,Y,N,N,Y,Shivering
2,3,3,Y,N,N,24-04-1999,N,Y,Y,N,N,Y,Head Ache
3,4,4,Y,N,N,15-11-2016,N,Y,Y,N,N,Y,Nausia
4,5,5,Y,N,N,13-11-1985,N,Y,Y,N,N,Y,Sickness
5,6,6,Y,N,N,28-12-2018,N,Y,Y,N,N,Y,Vertigo
6,7,7,Y,N,N,14-01-2014,N,Y,Y,N,N,Y,Head Ache
7,8,8,Y,N,N,08-11-2014,N,Y,Y,N,N,Y,Nausia
8,9,9,Y,N,N,24-08-1983,N,Y,Y,N,N,Y,Sickness
9,10,10,Y,N,N,01-07-2002,N,Y,Y,N,N,Y,"Vertigo, Nausia"


In [2]:
symptoms = {
    'Symptom1': {'Fever': ['Mild', 'Low', 'High']},
    'Symptom2': {'Cough': ['Mild', 'Low', 'High']},
    'Symptom3': {'Cold': ['Mild', 'Low', 'High']}
}

print(symptoms)

{'Symptom1': {'Fever': ['Mild', 'Low', 'High']}, 'Symptom2': {'Cough': ['Mild', 'Low', 'High']}, 'Symptom3': {'Cold': ['Mild', 'Low', 'High']}}


STEP 2


In [3]:
# Extract unique symptoms from the "Other_Symptoms" column
other_symptoms = df['Other_Symptoms'].str.split(',').explode().str.strip().unique()

# Prepare a list to track losses (symptoms not found in the dictionary)
loss = []

# Check each unique symptom in "Other_Symptoms" and compare with the dictionary
for symptom in other_symptoms:
    found = False
    # Check if symptom exists in any of the symptom groups in the dictionary
    for symptom_group, symptom_info in symptoms.items():
        if symptom in symptom_info:
            found = True
            break
    # If not found, add it to the dictionary and log it as loss
    if not found:
        loss.append(symptom)
        new_symptom_group = f'Symptom{len(symptoms) + 1}'
        # symptoms[new_symptom_group] = {symptom: ['Mild', 'Low', 'High']}
        symptoms[new_symptom_group] = {symptom:None}

# Print the updated symptoms dictionary
print("Updated Symptoms Dictionary:")
for symptom_group, symptom_info in symptoms.items():
    print(f"{symptom_group}: {symptom_info}")

# Print the loss
print("\nLoss (new symptoms added to the dictionary):")
print(loss)

Updated Symptoms Dictionary:
Symptom1: {'Fever': ['Mild', 'Low', 'High']}
Symptom2: {'Cough': ['Mild', 'Low', 'High']}
Symptom3: {'Cold': ['Mild', 'Low', 'High']}
Symptom4: {'Body Ache': None}
Symptom5: {'Shivering': None}
Symptom6: {'Head Ache': None}
Symptom7: {'Nausia': None}
Symptom8: {'Sickness': None}
Symptom9: {'Vertigo': None}
Symptom10: {'Vomitting': None}
Symptom11: {'Nose Bleed': None}

Loss (new symptoms added to the dictionary):
['Body Ache', 'Shivering', 'Head Ache', 'Nausia', 'Sickness', 'Vertigo', 'Vomitting', 'Nose Bleed']


In [4]:
# Calculate and print loss as a number
total_columns = len(symptoms)
loss_number = len(loss) / total_columns if total_columns > 0 else 0
print("\nLoss as a number:", loss_number)


Loss as a number: 0.7272727272727273


STEP 3


In [5]:
for symptom_group, symptom_info in symptoms.items():
    for symptom, severity in symptom_info.items():
        if severity is None or isinstance(severity, str):
            # Update symptoms without severity levels
            symptoms[symptom_group][symptom] = ['Mild', 'Low', 'High']

# Print the updated dictionary
print("Updated Symptoms Dictionary with Severity Levels:")
for symptom_group, symptom_info in symptoms.items():
    print(f"{symptom_group}: {symptom_info}")

Updated Symptoms Dictionary with Severity Levels:
Symptom1: {'Fever': ['Mild', 'Low', 'High']}
Symptom2: {'Cough': ['Mild', 'Low', 'High']}
Symptom3: {'Cold': ['Mild', 'Low', 'High']}
Symptom4: {'Body Ache': ['Mild', 'Low', 'High']}
Symptom5: {'Shivering': ['Mild', 'Low', 'High']}
Symptom6: {'Head Ache': ['Mild', 'Low', 'High']}
Symptom7: {'Nausia': ['Mild', 'Low', 'High']}
Symptom8: {'Sickness': ['Mild', 'Low', 'High']}
Symptom9: {'Vertigo': ['Mild', 'Low', 'High']}
Symptom10: {'Vomitting': ['Mild', 'Low', 'High']}
Symptom11: {'Nose Bleed': ['Mild', 'Low', 'High']}


In [6]:
total_columns = len(symptoms)
loss_number = len(loss) / total_columns if total_columns > 0 else 0
print("\nLoss as a number:", loss_number)


Loss as a number: 0.7272727272727273


In [7]:
import json
with open("symptoms_dict.json", 'w') as f:
        json.dump(symptoms, f, indent=4)

STEP 4

In [8]:
import pandas as pd
import json
import xml.etree.ElementTree as ET
import re

class SymptomsParser:
    def __init__(self, dict_file='symptoms_dict.json'):
        self.symptoms_dict = {}
        self.dict_file = dict_file
        self.load_dictionary()

    # Method to read data from multiple formats
    def read_data(self, file_path):
        if file_path.endswith('.csv'):
            data = pd.read_csv(file_path)
        elif file_path.endswith('.tsv'):
            data = pd.read_csv(file_path, delimiter='\t')
        elif file_path.endswith('.json'):
            data = pd.read_json(file_path)
        elif file_path.endswith('.xml'):
            data = self.parse_xml_to_dataframe(file_path)
        else:
            raise ValueError("Unsupported file format")
        return data

    # Custom XML parser to DataFrame
    @staticmethod
    def parse_xml_to_dataframe(file_path):
        tree = ET.parse(file_path)
        root = tree.getroot()
        data = []
        for patient in root.findall('./Patient'):
            row = {}
            for child in patient:
                row[child.tag] = child.text
            data.append(row)
        return pd.DataFrame(data)

    # Method to enhance the dictionary by adding missing symptoms from the dataset
    def enhance_dictionary(self, data, symptom_col='Other_Symptoms'):
        # Loop through each row in the symptom column and extract symptoms
        for symptoms in data[symptom_col].dropna():
            # Split symptoms by comma (assuming multiple symptoms are comma-separated)
            for symptom in symptoms.split(','):
                symptom = symptom.strip()  # Remove any extra spaces

                # Check if this symptom is already in the dictionary
                found = False
                for symptom_group in self.symptoms_dict.values():
                    if symptom in symptom_group:
                        found = True
                        break

                # If not found, add it with default severities
                if not found:
                    symptom_key = f'Symptom{len(self.symptoms_dict) + 1}'
                    self.symptoms_dict[symptom_key] = {symptom: ['Mild', 'Low', 'High']}

        return self.symptoms_dict

    # Method to print the data based on the available columns in the dictionary
    def print_data_based_on_dict(self, data, symptom_col='Other_Symptoms'):
        available_symptoms = [symptom for group in self.symptoms_dict.values() for symptom in group]
        filtered_data = data[data[symptom_col].apply(lambda x: any(symptom in x for symptom in available_symptoms) if pd.notnull(x) else False)]
        print("Filtered Data Based on Available Symptoms in Dictionary:")
        print(filtered_data)

    # Method to dump the dictionary to a file
    def dump_dictionary(self):
        with open(self.dict_file, 'w') as f:
            json.dump(self.symptoms_dict, f, indent=4)
        print(f"Dictionary has been dumped to {self.dict_file}")

    # Method to load the dictionary from a file
    def load_dictionary(self):
        try:
            with open(self.dict_file, 'r') as f:
                self.symptoms_dict = json.load(f)
        except FileNotFoundError:
            print(f"No dictionary found at {self.dict_file}. Initializing with an empty dictionary.")
            self.symptoms_dict = {}

    # Method to update the dictionary after manual editing
    def manual_update(self):
        symptom = input("Enter the symptom you want to add: ")
        severity_levels = input("Enter severity levels (comma-separated, e.g., Mild, Low, High): ")
        severity_levels = [level.strip() for level in severity_levels.split(',')]

        # Check if symptom already exists
        symptom_exists = any(symptom in group for group in self.symptoms_dict.values())
        if symptom_exists:
            print(f"Symptom '{symptom}' already exists in the dictionary.")
        else:
            symptom_key = f'Symptom{len(self.symptoms_dict) + 1}'
            self.symptoms_dict[symptom_key] = {symptom: severity_levels}
            print(f"Added '{symptom}' with levels {severity_levels} to the dictionary.")
        self.dump_dictionary()

    # Main method to process the data
    def process_data(self, file_path, symptom_col='Other_Symptoms'):
        # Step 1: Read data
        data = self.read_data(file_path)

        # Step 2: Print data as per the available columns in the dictionary
        self.print_data_based_on_dict(data, symptom_col=symptom_col)

        # Step 3: Enhance dictionary with new symptoms found in the dataset
        self.enhance_dictionary(data, symptom_col=symptom_col)

        # Step 4: Reprint data based on the updated dictionary
        self.print_data_based_on_dict(data, symptom_col=symptom_col)

        # Step 5: Dump the enhanced dictionary to a file for manual editing
        self.dump_dictionary()




# Usage example:
if __name__ == "__main__":
    # Instantiate the parser
    parser = SymptomsParser(dict_file='symptoms_dict.json')

    # Process a data file (CSV, TSV, JSON, XML supported)
    file_path = 'data.csv'  # Replace with your actual file path
    parser.process_data(file_path, symptom_col='Other_Symptoms')
    choice = input("Do you want to manually update the dictionary : [Y,N]")
    if(choice == "Y"):
        parser.manual_update()
    else:
        pass

Filtered Data Based on Available Symptoms in Dictionary:
    SrNo  Patient_Id Fever_Mild Fever_Low Fever_High         DOB Cough_Mild  \
0      1           1          Y         N          N  02-06-1997          N   
1      2           2          Y         N          N  28-06-2017          N   
2      3           3          Y         N          N  24-04-1999          N   
3      4           4          Y         N          N  15-11-2016          N   
4      5           5          Y         N          N  13-11-1985          N   
5      6           6          Y         N          N  28-12-2018          N   
6      7           7          Y         N          N  14-01-2014          N   
7      8           8          Y         N          N  08-11-2014          N   
8      9           9          Y         N          N  24-08-1983          N   
9     10          10          Y         N          N  01-07-2002          N   
10    11          11          Y         N          N   4-03-2003          